### This file is run only once to download the data from its source location and store in S3

In [3]:
import pandas as pd
import re
import boto3


In [2]:
!mkdir ../data
!wget -O ../data/WISDM_ar_latest.tar.gz http://www.cis.fordham.edu/wisdm/includes/datasets/latest/WISDM_ar_latest.tar.gz
!tar -zxf ../data/WISDM_ar_latest.tar.gz -C ../data 

--2020-03-22 21:05:02--  http://www.cis.fordham.edu/wisdm/includes/datasets/latest/WISDM_ar_latest.tar.gz
Resolving www.cis.fordham.edu (www.cis.fordham.edu)... 150.108.68.29
Connecting to www.cis.fordham.edu (www.cis.fordham.edu)|150.108.68.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11404612 (11M) [application/x-gzip]
Saving to: ‘../data/WISDM_ar_latest.tar.gz’

../data/WISDM_ar_la 100%[===================>]  10.88M  8.23MB/s    in 1.3s    

2020-03-22 21:05:03 (8.23 MB/s) - ‘../data/WISDM_ar_latest.tar.gz’ saved [11404612/11404612]



In [13]:
!aws s3api create-bucket --bucket sagemaker-wisdm --region us-east-1

{
    "Location": "/sagemaker-wisdm"
}


In [4]:
bucket = "sagemaker-wisdm"
data_location = 's3://{}'.format(bucket)

s3 = boto3.resource('s3')

In [5]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv('../data/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt', header= None, names = columns)

df['z-axis'] = [re.sub(';','', str(df.loc[row,'z-axis'])) for row in range(len(df))]    

### Data is split into 12 files to simulate di

In [9]:
length = int(len(df)/12)

for i in range(12):
    init=int(i*length)
    fin = int(init+length)
    prefix = '../'
    fname = 'data/df' + str(i) + '.csv'
    df[init:fin].to_csv(str(prefix+fname), index=False, header=True)
    s3.Bucket(bucket).upload_file(str(prefix+fname), fname)
    print(fname)

data/df0.txt
data/df1.txt
data/df2.txt
data/df3.txt
data/df4.txt
data/df5.txt
data/df6.txt
data/df7.txt
data/df8.txt
data/df9.txt
data/df10.txt
data/df11.txt
